# Script for Generating Training Data from Frames using PIPs
https://github.com/aharley/pips

## Initial set up
Connection to MyDrive, requirements, imports, downloading models weights

In [1]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing requirements
!pip install imageio==2.19.3
!pip install opencv-python==4.6.0.66
!pip install einops==0.4.1
!pip install scikit-learn==1.1.1
!pip install matplotlib==3.5.1
!pip install protobuf==3.20.0
!pip install fire==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 50.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 62.5 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.58.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-translate 3.8.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/drive_folder/pips')
import time
import numpy as np
import io
import os
from PIL import Image
import cv2
import saverloader
from nets.pips import Pips
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gc
import imageio.v2 as imageio
from csv import writer

In [2]:
# downloading their reference model
!bash /content/drive/MyDrive/drive_folder/pips/get_reference_model.sh

downloading the model from dropbox...
--2023-02-03 09:26:35--  https://www.dropbox.com/s/hbo7ns4vfx1sejp/reference_model.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/hbo7ns4vfx1sejp/reference_model.tar.gz [following]
--2023-02-03 09:26:36--  https://www.dropbox.com/s/raw/hbo7ns4vfx1sejp/reference_model.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26d5305c6ed76be4a6d6b17530.dl.dropboxusercontent.com/cd/0/inline/B1zYnC75jH2ka3xAnK4ffiHU0bmAw7IEpSCt6M42mxMP8y_HvJuSWuumy7oZt0XBJZRBNyJaT_2rKQTMgxRUPL8fq88Fi1kkFY0Ubma9jdnIXToWosnB57qaejKmzHjdXPsA_RQUgCXE-4TlE9BCEyKLawGEHzBnmfGt-IjjTqZUZw/file# [following]
--2023-02-03 09:26:36--  https://uc26d5305c6ed76be4a6d6b17530.dl.dropboxusercontent.com/cd/0/inline/B1zYnC75j

## Loading directory with data frames to generate training data from

In [ ]:
DATA_DIR = "frames" # specify the name of the data folder located in drive
!unzip -d "$DATA_DIR"/ /content/drive/MyDrive/"$DATA_DIR".zip # unziping frames

## Generating training data

### PIPs implementation

In [20]:
def create_model(model_weights):
  model = Pips(stride=4).cuda()
  parameters = list(model.parameters())
  _ = saverloader.load(model_weights, model)
  global_step = 0
  model.eval()
  return model


def run_model(model, rgbs, N, split):
    rgbs = rgbs.cuda().float() # B, S, C, H, W

    B, S, C, H, W = rgbs.shape
    rgbs_ = rgbs.reshape(B*S, C, H, W)
    H_, W_ = 360, 640
    rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
    H, W = H_, W_
    rgbs = rgbs_.reshape(B, S, C, H, W)

    # pick N points to track; we'll use a uniform grid
    N_ = np.sqrt(N).round().astype(np.int32)
    grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
    grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
    grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
    xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
    _, S, C, H, W = rgbs.shape

    print_stats('rgbs', rgbs)

    # splitting grid of points into tensors of size 1500 along dim=1
    if split:
       xy_split = torch.split(xy, 1500, dim=1)
       preds_split = []
       vis_split = []

      # for each splitted point compute trajs
       for i in range(len(xy_split)):
         preds, preds_anim, vis, stats = model(xy_split[i], rgbs, iters=6)
         preds_split.append(preds[-1])
         vis_split.append(vis[-1])

       # put trajs back together
       trajs_e = torch.cat(preds_split, 2)
       vis_e = torch.cat(vis_split, 1).unsqueeze(0)
       

    else:
      preds, preds_anim, vis, stats = model(xy, rgbs, iters=6)
      # preds is a list of torch tensors 
      trajs_e = preds[-1] # tensor of shape (frames, tracking points, 2)
      vis_e = vis

    print_stats('vis_e', vis_e)
    return trajs_e[:,-1,:,:], vis_e[:,-1,:]

def generate_training_data(pips_model, video_name, n, split=True):
    model = pips_model
    
    ## Choose hyps
    B = 1
    S = 8
    N = n**2 # number of points to track

    # Get frames of a file
    filenames = glob.glob(DATA_DIR+'/content/frames/'+video_name+'/frames/*.jpg')
    filenames = sorted(filenames)

    # Remove every second frame from a list
    n = 2
    del filenames[n - 1::n]

    max_iters = len(filenames)//S # run each unique subsequence
    iters = 100
    global_step = 0

    # Run model each of 8 frames
    while global_step < iters:
        global_step += 1

        if (global_step < 100) or (global_step>(max_iters-100)):
          continue
        
        try:
            rgbs = []
            sample_id = video_name[-11:]+"_"+str((global_step-1)*S)
            print("sample {}: step {}/{}".format(sample_id, global_step, max_iters))

            # skip generating this sample if already in the log list
            if sample_id in LOG_LIST:
                  print(sample_id+" already in log list")
                  continue
            for s in range(S):
                frame_num = (global_step-1)*S+s
                fn = filenames[frame_num]
                if s==0:
                    print('start frame', fn)
                im = imageio.imread(fn)
                im = im.astype(np.uint8)
                im = torch.from_numpy(im).permute(2,0,1)
                rgbs.append(im)
            rgbs = torch.stack(rgbs, dim=0).unsqueeze(0) # 1, S, C, H, W

            with torch.no_grad():
                trajs_e, vis_e = run_model(model, rgbs, N, split)

            # store img0 and img1 (frame 1 and 8) and trajs_e, vis_e
            save_data(sample_id, rgbs[0][0], rgbs[0][1], rgbs[0][3], rgbs[0][-1], trajs_e, vis_e)

        except FileNotFoundError as e:
            print('error', e)

def img_to_array(tensor_img):
   x = tensor_img.permute(1, 2, 0).numpy() # from tensor to numpy array
   x = cv2.resize(x, (240, 120)) # resizing image
   return x

def save_data(sample_id, frame0, frame1, frame3, frame7, trajs, vis):
  """
  Saving
  - coordinates to COORDS_DIR as .npy files
  - visibility of coordinates to VIS_DIR as .npy files
  - 0th frames to FRAME0_DIR directory as .npy files
  - 7th frames to FRAME7_DIR directory as .npy files
  
  - Coordinates, visibility and frames are mapped with sample_id, all sample_ids are stored in sample_ids.txt
  """
  img0 = img_to_array(frame0) # transform 1st frame to array
  img1 = img_to_array(frame1) # transform 2nd frame to array
  img3 = img_to_array(frame3) # transform 2nd frame to array
  img7 = img_to_array(frame7) # tranform last frame (8th) to array
  coords = trajs.cpu().numpy()
  vis = vis.cpu().numpy()

  # Store coords, frame0, frame1
  np.save(COORDS_DIR+sample_id, coords)
  np.save(VIS_DIR+sample_id, vis)
  np.save(FRAME0_DIR+sample_id, img0)
  np.save(FRAME1_DIR+sample_id, img1)
  np.save(FRAME3_DIR+sample_id, img3)
  np.save(FRAME7_DIR+sample_id, img7)

  # Add sample_id to log list
  LOG_LIST.append(sample_id)
  save_log(LOG_LIST)

def save_log(log_list):
  with open(LOG_FILE, "w") as outfile:
    outfile.write("\n".join(log_list))

In [9]:
model = create_model('reference_model')

reading ckpt from reference_model
...found checkpoint reference_model/model-000100000.pth


### Creating directories and running generation of data

In [22]:
# create a folder to store training data in
!rm -r training_data
!mkdir training_data
!mkdir training_data/frame0/
!mkdir training_data/frame1/
!mkdir training_data/frame3/
!mkdir training_data/frame7/
!mkdir training_data/coords/
!mkdir training_data/vis/
!touch training_data/sample_ids.txt
PATH = ""

In [ ]:
# if we want to use an old training data folder
# cp from drive to colab memory and unzip
TRAIN_DATA_DIR = "training_data_14122022" # specify the name of the data folder located in drive
!unzip -d "$TRAIN_DATA_DIR"/ /content/drive/MyDrive/"$TRAIN_DATA_DIR".zip # unziping frames
PATH = TRAIN_DATA_DIR+"/content/"

In [23]:
TRANING_DATA_DIR = PATH+"training_data/"
COORDS_DIR = PATH+"training_data/coords/"
VIS_DIR = PATH+"training_data/vis/"
FRAME0_DIR= PATH+"training_data/frame0/"
FRAME1_DIR= PATH+"training_data/frame1/"
FRAME3_DIR= PATH+"training_data/frame3/"
FRAME7_DIR= PATH+"training_data/frame7/"
LOG_FILE = PATH+"training_data/sample_ids.txt"
LOG_LIST = open(LOG_FILE).read().splitlines()

In [24]:
generate_training_data(model, 'Tai chiKunlun Si Xiang Quan Demonstrated by Huang Shuanqing--CR4xjdQbkc', n=64)
# generate_training_data(model, 'Tai chi陈式太极拳五十六式-vB8XTJfV4rY', n=64)

sample -CR4xjdQbkc_792: step 100/403
start frame frames/content/frames/Tai chiKunlun Si Xiang Quan Demonstrated by Huang Shuanqing--CR4xjdQbkc/frames/frame_01585.jpg
rgbs (float32) min = 0.00, mean = 169.58, max = 255.00 torch.Size([1, 8, 3, 360, 640])
vis_e (float32) min = -6.82, mean = 6.42, max = 12.08 torch.Size([1, 8, 4096])


In [ ]:
generate_training_data(model, 'Tai chiOld Frame Wǔ Family Taiji Quan,Rare Exercises and Tui Shou-Sh2siPvCPo0', n=64)
generate_training_data(model, 'Tai chiKunlun Si Xiang Quan Demonstrated by Huang Shuanqing--CR4xjdQbkc', n=64)
generate_training_data(model, 'Tai chiSun Style Tai Ji Quan - Xing Yi Sword form-S9MN1i6Fejc', n=64)
generate_training_data(model, 'Tai chi世界太極拳冠軍,邱慧芳示範,太極拳16武-fL8f6X3JPzE', n=64)
generate_training_data(model, 'Tai chiYang Taijiquan 16 vorm-KjMxdYJOwqI', n=64)
generate_training_data(model, 'Tai chiTrần thức giản hóa do Vs - Thiều Ngọc Sơn diễn luyện.-MGYF1aDwUKg', n=64)
generate_training_data(model, 'Tai chiJin Gang Kwan Dao-3zIcD0hIpfc', n=64)
generate_training_data(model, 'Tai chi武当丹剑.flv-42PJ4MlxWKA', n=64)
generate_training_data(model, 'Tai chiWudang Taichi 18 Xuanwu Style 武当18式太極拳（玄武派） Master Xu Gu 虚谷师父-Mo87mOAnAio', n=64)
generate_training_data(model, 'Tai chiTai Chi 32 Yang Stil Schwertform  _ 32 Step Yang Style Tai Chi Sword Routine-asL3ZyuNeB0', n=64)

In [ ]:
generate_training_data(model, 'Tai chiДаосский тайцзицюань школы Лун Мэнь Первая форма 1-й Раздел Ли Фа Цзюнь-8DemshD_8a4', n=64)
generate_training_data(model, 'Tai chiWuDang Tai Ji 28  二十八式 2009拍.flv-BtuIV3k5O6Q', n=64)
generate_training_data(model, 'Tai chi吳式太極快架-KMO1BluPtU4', n=64)
generate_training_data(model, 'Tai chiWudang Taijiquan Forms - Cheng Tinhung (鄭天熊) lineage-zxsGZCdT7Ns', n=64)
generate_training_data(model, 'Tai chiTai Chi sword form-sm6-dbG5Rho', n=64)
generate_training_data(model, 'Tai chizhaobao Taichi 24 刘钢 示范 武当赵堡和式太极拳二十四式简化太极拳-FmJ33H476pI', n=64)
generate_training_data(model, 'Tai chi陈式太极拳五十六式-vB8XTJfV4rY', n=64)
generate_training_data(model, 'Tai chi64 式太極拳練習（第一段），090927。-K7td6IVvS8w', n=64)
generate_training_data(model, 'Tai chi56式夕陽美功夫扇-uOw-z7CR7x8', n=64)
generate_training_data(model, 'Tai chiTai Chi for Osteoporosis Front Demo-AUzpWyGi8Gw', n=64)

In [ ]:
generate_training_data(model, 'Tai chichen 56-l0aKiGrTUGw', n=64)
generate_training_data(model, 'Tai chi陳氏太極拳：老架一路(上)王西安.flv-CA9-8CydW4A', n=64)
generate_training_data(model, 'Tai chiTaijiquan Tai Chi Pai Lin part 2-6f6nS3PCeQM', n=64)

In [ ]:
# create a list of file names to go through
videos_list = [f for f in os.listdir(DATA_DIR+'/content/frames/') if not f.startswith('.')]
print(videos_list)

In [ ]:
# iterate through the videos list and generate training data files
for x in range(len(videos_list)):
  print("Now generating training data for {}, {} out of {}.".format(videos_list[x], x, len(videos_list)))
  generate_training_data(model, videos_list[x], 100)

### Zip data and store in Drive

In [25]:
from datetime import date
TODAY = date.today()

# zip training data
!cd /content/"$PATH"training_data && zip -r /content/training_data_1_2frames"$TODAY".zip .
# cp NumPy zip file into drive
!cp training_data_1_2frames"$TODAY".zip /content/drive/MyDrive/

  adding: vis/ (stored 0%)
  adding: vis/-CR4xjdQbkc_792.npy (deflated 12%)
  adding: frame0/ (stored 0%)
  adding: frame0/-CR4xjdQbkc_792.npy (deflated 45%)
  adding: sample_ids.txt (stored 0%)
  adding: frame7/ (stored 0%)
  adding: frame7/-CR4xjdQbkc_792.npy (deflated 45%)
  adding: frame3/ (stored 0%)
  adding: frame3/-CR4xjdQbkc_792.npy (deflated 46%)
  adding: coords/ (stored 0%)
  adding: coords/-CR4xjdQbkc_792.npy (deflated 11%)
  adding: frame1/ (stored 0%)
  adding: frame1/-CR4xjdQbkc_792.npy (deflated 44%)
